In [ ]:
import PyPDF2
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Extract text from PDF
def extract_text_from_pdf(file_path):
    pdf_text = ""
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:  # Check if text was extracted
                pdf_text += page_text + " "
    return pdf_text

# Step 2: Split the text into sentences
def split_into_sentences(text):
    if not text.strip():  # Check if the text is empty or whitespace
        return []
    sentences = re.split(r'(?<=[.!?]) +', text)
    return sentences

# Step 3: Find similar sentences using cosine similarity
def find_similar_statements(target_statement, sentences):
    if not sentences:
        print("No sentences found in the PDF.")
        return []
    
    vectorizer = TfidfVectorizer().fit_transform([target_statement] + sentences)
    vectors = vectorizer.toarray()
    cosine_matrix = cosine_similarity(vectors)
    target_vector = cosine_matrix[0]  # First row corresponds to the target statement

    # Rank sentences by similarity (skip the first as it is the target itself)
    similar_sentences = sorted(
        enumerate(target_vector[1:], 1), key=lambda x: x[1], reverse=True
    )
    return [(sentences[i], score) for i, score in similar_sentences]

# Example usage
pdf_path = 'path/to/your/pdf_file.pdf'
target = 'Your target statement to compare against.'

text = extract_text_from_pdf(pdf_path)
if not text:
    print("Failed to extract text from the PDF.")
else:
    sentences = split_into_sentences(text)
    results = find_similar_statements(target, sentences)

    # Print top 5 most similar sentences
    if results:
        for sentence, similarity in results[:5]:
            print(f"Similarity: {similarity:.2f} - Sentence: {sentence}")
    else:
        print("No similar statements found.")


In [ ]:
import PyPDF2
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Extract text from PDF
def extract_text_from_pdf(file_path):
    pdf_text = ""
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            pdf_text += page.extract_text() + " "
    return pdf_text

# Step 2: Split the text into sentences
def split_into_sentences(text):
    sentences = re.split(r'(?<=[.!?]) +', text)
    return sentences

# Step 3: Find similar sentences using cosine similarity
def find_similar_statements(target_statement, sentences):
    vectorizer = TfidfVectorizer().fit_transform([target_statement] + sentences)
    vectors = vectorizer.toarray()
    cosine_matrix = cosine_similarity(vectors)
    target_vector = cosine_matrix[0]  # First row corresponds to the target statement

    # Rank sentences by similarity (skip the first as it is the target itself)
    similar_sentences = sorted(
        enumerate(target_vector[1:], 1), key=lambda x: x[1], reverse=True
    )
    return [(sentences[i], score) for i, score in similar_sentences]

# Example usage
pdf_path = 'path/to/your/pdf_file.pdf'
target = 'Your target statement to compare against.'

text = extract_text_from_pdf(pdf_path)
sentences = split_into_sentences(text)
results = find_similar_statements(target, sentences)

# Print top 5 most similar sentences
for sentence, similarity in results[:5]:
    print(f"Similarity: {similarity:.2f} - Sentence: {sentence}")
